# 卷積的矩陣化

這樣的做法主要是能提升卷積計算的效能。

In [1]:
import numpy as np

# im2row 的第一個版本，使用雙層迴圈遍歷空間維度
# 這個版本的效率較低，因為 Python 的迴圈性能不佳，實際使用中建議改用基於索引的版本 (im2row_indices)
# 參數：x: 輸入圖片 (N, C, H, W), kH: 卷積核高度, kW: 卷積核寬度, S: 步長 (Stride)
def im2row(x, kH, kW, S=1):
    # N: 樣本數, C: 通道數, H: 影像高度, W: 影像寬度
    N, C, H, W = x.shape
    
    # 計算輸出特徵圖 (Feature Map) 的空間維度
    oH = (H - kH) // S + 1
    oW = (W - kW) // S + 1
    
    # 初始化一個空的矩陣來存放結果
    # 列數 (Rows)：總共會取出的區塊數量 (N * 輸出高度 * 輸出寬度)
    # 欄數 (Cols)：每個區塊攤平後的總像素點 (卷積核高 * 卷積核寬 * 通道數)
    row = np.empty((N * oH * oW, kH * kW * C))
    
    oSize = oH * oW  # 單張圖片產生的區塊總數
    
    # 透過雙層迴圈遍歷所有輸出的座標點 (h, w)
    for h in range(oH):
        hS = h * S           # 窗口在原始影像上的垂直起始位置
        hS_kH = hS + kH      # 窗口在原始影像上的垂直結束位置
        h_start = h * oW     # 這是為了定位目前在 oSize 中的相對索引
        
        for w in range(oW):
            wS = w * S       # 窗口在原始影像上的水平起始位置
            
            # 1. 擷取區塊 (Patch)：從所有樣本中取出該位置的窗口
            # patch 的維度維 (N, C, kH, kW)
            patch = x[:, :, hS:hS_kH, wS:wS + kW]
            
            # 2. 攤平並填入矩陣：
            # 利用步進切片 (Step Slicing) [h_start+w::oSize] 
            # 將 N 個樣本的相同位置區塊，分散填入對應的 Row
            # 例如：row[0] 是第一個樣本的區塊1, row[oSize] 是第二個樣本的區塊1
            row[h_start + w::oSize, :] = np.reshape(patch, (N, -1))
            
    return row

In [2]:
# x = np.arange(15).reshape(3, 1, 1, 5)  # (N=3, C=1, H=1, W=5) 的輸入影像
# print("Input Image (N, C, H, W):")
# print(x)

# x_row = im2row(x, 1, 4, S=1)
# print("\nOutput of im2row (N*oH*oW, kH*kW*C):")
# print(x_row)

In [3]:
x = np.arange(18).reshape(1, 2, 3, 3)  # (N=1, C=2, H=3, W=3)
print("Input Image (N, C, H, W):")
print(x)

x_row = im2row(x, 2, 2, S=1)
print("\nOutput of im2row (N*oH*oW, kH*kW*C):")
print(x_row)

Input Image (N, C, H, W):
[[[[ 0  1  2]
   [ 3  4  5]
   [ 6  7  8]]

  [[ 9 10 11]
   [12 13 14]
   [15 16 17]]]]

Output of im2row (N*oH*oW, kH*kW*C):
[[ 0.  1.  3.  4.  9. 10. 12. 13.]
 [ 1.  2.  4.  5. 10. 11. 13. 14.]
 [ 3.  4.  6.  7. 12. 13. 15. 16.]
 [ 4.  5.  7.  8. 13. 14. 16. 17.]]


In [4]:
# x = np.arange(36).reshape(2, 2, 3, 3)  # (N=2, C=2, H=3, W=3) 的輸入影像
# print("Input Image (N, C, H, W):")
# print(x)

# x_row = im2row(x, 2, 2, S=1)
# print("\nOutput of im2row (N*oH*oW, kH*kW*C):")
# print(x_row)

In [5]:
def conv_forward(X, K, S=1, P=0):
    """
    卷積層的前向傳播 (Forward Pass)
    """
    N, C, H, W  = X.shape      # 輸入資料的維度 (Batch, Channel, Height, Width)
    F, C_k, kH, kW = K.shape   # 卷積核 (Kernel/Filter) 維度
    
    # 處理補零 (Padding)
    if P == 0:
        X_pad = X
    else:
        # 在 H 與 W 維度進行常數補零
        X_pad = np.pad(X, ((0, 0), (0, 0), (P, P), (P, P)), 'constant')
   
    # 核心優化：將圖片區塊轉換成矩陣行 (im2row)，方便後續進行矩陣運算 (Matrix Multiplication)
    X_row = im2row(X_pad, kH, kW, S)
    
    # 將卷積核攤平 (Flatten) 並轉置，準備進行點積
    K_col = K.reshape(K.shape[0], -1).transpose()    
    
    # 執行矩陣相乘，這是卷積運算效能最高的核心步驟
    Z_row = np.dot(X_row, K_col)
    
    # 計算輸出特徵圖 (Feature Map) 的空間維度
    oH = (X_pad.shape[2] - kH) // S + 1
    oW = (X_pad.shape[3] - kW) // S + 1
    
    # 將結果從矩陣形式 Reshape 回 (N, C, H, W) 格式
    Z = Z_row.reshape(N, oH, oW, -1)
    Z = Z.transpose(0, 3, 1, 2) # 調整維度順序為 (Batch, Filter, Height, Width)
    return Z

In [6]:
import numpy as np

# 假設之前的 im2row 與 conv_forward 已定義
x = np.arange(9).reshape(1, 1, 3, 3) + 1  # 數值 1~9
k = np.arange(4).reshape(1, 1, 2, 2) + 1  # 數值 1~4

print("--- 輸入影像 (Input) ---")
print(x[0, 0])

print("\n--- 卷積核 (Kernel) ---")
print(k[0, 0])

# 呼叫 im2row (輸入, 卷積核高, 卷積核寬, 步長)
x_row = im2row(x, 2, 2, S=1)

print("\n--- im2row 後的矩陣形式 (X_row) ---")
# 每一列代表一個滑動窗口
print(x_row)

z = conv_forward(x, k)

print("\n--- 輸出結果 (Output Z) ---")
print("Shape:", z.shape)
print(z[0, 0])

--- 輸入影像 (Input) ---
[[1 2 3]
 [4 5 6]
 [7 8 9]]

--- 卷積核 (Kernel) ---
[[1 2]
 [3 4]]

--- im2row 後的矩陣形式 (X_row) ---
[[1. 2. 4. 5.]
 [2. 3. 5. 6.]
 [4. 5. 7. 8.]
 [5. 6. 8. 9.]]

--- 輸出結果 (Output Z) ---
Shape: (1, 1, 2, 2)
[[37. 47.]
 [67. 77.]]


In [7]:
import numpy as np

# 建立輸入資料 (Input Tensor)，維度為 (N=2, C=2, H=3, W=3)
x = np.arange(36).reshape(2, 2, 3, 3)

# 建立卷積核 (Kernel)，維度為 (F=2, C=2, kH=2, kW=2)
k = np.arange(16).reshape(2, 2, 2, 2)

# 執行卷積的前向傳播運算
z = conv_forward(x, k)

# 印出輸出張量的維度 (Shape)
print(z.shape)

# 印出運算後的數值結果
print(z)

(2, 2, 2, 2)
[[[[ 268.  296.]
   [ 352.  380.]]

  [[ 684.  776.]
   [ 960. 1052.]]]


 [[[ 772.  800.]
   [ 856.  884.]]

  [[2340. 2432.]
   [2616. 2708.]]]]


In [8]:
# 將矩陣化的 Row 向量還原回圖片格式（用於反向傳播時，將梯度還原回原始影像維度）
# 參數：dx_row: 從上一層傳回的梯度 (已經是 Row 形式), oH/oW: 輸出特徵圖的空間維度
#      kH/kW: 卷積核尺寸, S: 步長 (Stride)
# 返回值：dx: 還原回原始圖片維度的梯度
def row2im(dx_row, oH, oW, kH, kW, S):
    nRow, K2C = dx_row.shape[0], dx_row.shape[1]
    C = K2C // (kH * kW)  # 計算通道數 (Channels)
    N = nRow // (oH * oW)  # 計算樣本數 (Batch Size)
    oSize = oH * oW        # 輸出特徵圖的總像素點
    H = (oH - 1) * S + kH  # 計算還原後的原始高度
    W = (oW - 1) * S + kW  # 計算還原後的原始寬度
    dx = np.zeros([N, C, H, W])
    
    for i in range(oSize):
        # 根據步長 (Stride) 抓出 N 個樣本對應的行向量
        row = dx_row[i::oSize, :]         
        h_start = (i // oW) * S
        w_start = (i % oW) * S     
        # 將梯度累加回對應的圖片區域（處理重疊區域的梯度加總）
        dx[:, :, h_start:h_start+kH, w_start:w_start+kW] += row.reshape((N, C, kH, kW))
    return dx

# row2im 的另一個版本，邏輯相似但透過雙層迴圈處理空間維度
def row2im_v2(dx_row, oH, oW, kH, kW, S):
    nRow, K2C = dx_row.shape[0], dx_row.shape[1]
    C = K2C // (kH * kW)
    N = nRow // (oH * oW)    
    oSize = oH * oW  
    
    H = (oH - 1) * S + kH
    W = (oW - 1) * S + kW
    dx = np.zeros([N, C, H, W])
    for h in range(oH):
        hS = h * S
        hS_kH = hS + kH
        h_start = h * oW
        for w in range(oW):
            wS = w * S          
            # 根據目前的索引位置取出對應的 row
            row = dx_row[h_start+w::oSize, :]                    
            dx[:, :, hS:hS_kH, wS:wS+kW] += row.reshape(N, C, kH, kW)          
    return dx

# 卷積層的反向傳播 (Backward Pass)
# 參數：X_row: 前向傳播時轉換成 Row 的輸入資料, dZ: 上一層傳回的梯度, K: 卷積核參數,
#      oH/oW: 輸出特徵圖的空間維度, kH/kW: 卷積核的尺寸, S: 步長 (Stride), P: 補零 (Padding)
# 返回值：dX: 傳給前一層的梯度, dK: 卷積核參數的梯度, db: 偏差項的梯度
def conv_backward(X_row, dZ, K, oH, oW, kH, kW, S=1, P=0):
    F = dZ.shape[1]  
    # 將上一層傳回的梯度 (dZ) 進行轉置並攤平
    dZ_row = dZ.transpose(0, 2, 3, 1).reshape(-1, F)
    
    # 1. 計算關於卷積核參數的梯度 (dK)
    dK_col = np.dot(X_row.T, dZ_row) 
    dK_col = dK_col.transpose(1, 0)
    dK = dK_col.reshape(K.shape)
    
    # 2. 計算關於偏差項 (Bias) 的梯度 (db)
    db = np.sum(dZ, axis=(0, 2, 3))
    db = db.reshape(-1, F)
    
    # 3. 計算傳給前一層的梯度 (dX)
    K_col = K.reshape(K.shape[0], -1).transpose()  
    dX_row = np.dot(dZ_row, K_col.T)
    
    # 把梯度矩陣還原成原始圖片的形狀
    dX_pad = row2im(dX_row, oH, oW, kH, kW, S)
    
    # 如果當初有做 Padding，現在要把補零的部分切掉
    if P == 0:
        return dX_pad, dK, db
    return dX_pad[:, :, P:-P, P:-P], dK, db

In [9]:
# 設定輸入影像與卷積核的維度參數
H, W  = 4, 4      # 輸入特徵圖 (Input Feature Map) 的高與寬
kH, kW = 2, 2     # 卷積核 (Kernel/Filter) 的高與寬
oH, oW = 3, 3     # 輸出特徵圖 (Output Feature Map) 的高與寬
N  = 1            # 批次大小 (Batch Size)
C  = 3            # 輸入通道數 (Input Channels)
S  = 1            # 步長 (Stride)
P  = 0            # 填充 (Padding) 像素量
F  = 4            # 濾波器數量 (Number of Filters)，即輸出通道數

# 建立來自下一層傳回的梯度 dZ (Loss 對輸出的偏微分)
dZ = np.arange(N*F*oH*oW).reshape(N, F, oH, oW)

# 建立原始輸入資料 X
X = np.arange(N*C*H*W).reshape(N, C, H, W)

# 處理 Padding (填充)
if P == 0:
    X_pad = X     # 若無填充則維持原樣
else:
    # 在高 (H) 與 寬 (W) 的維度補零 (Zero Padding)
    X_pad = np.pad(X, ((0, 0), (0, 0), (P, P), (P, P)), 'constant')

# 建立卷積核權重 K (Weights)
K = np.arange(F*C*kH*kW).reshape(F, C, kH, kW)

# 將影像矩陣展開為 Row 形式 (im2row)，方便進行矩陣乘法運算
X_row = im2row(X_pad, kH, kW, S)

# 執行卷積層的反向傳播運算
# 計算出：
# dX: 對輸入的梯度 (傳回前一層)
# dW: 對權重的梯度 (用來更新濾波器參數)
# db: 對偏置 (Bias) 的梯度
dX, dW, db = conv_backward(X_row, dZ, K, oH, oW, kH, kW, S, P)   

# 印出結果進行維度檢查與數值驗證
print(dX.shape)
print("dX[0,0,:,:]:", dX[0,0,:,:]) # 檢視第一個樣本、第一個通道的輸入梯度
print(dW.shape)
print("dW[0,0,:,:]:", dW[0,0,:,:]) # 檢視第一個濾波器、第一個通道的權重梯度
print(db.shape)
print("db:", db)                  # 印出偏置項梯度

(1, 3, 4, 4)
dX[0,0,:,:]: [[1512. 3150. 3298. 1718.]
 [3348. 6968. 7280. 3788.]
 [3804. 7904. 8216. 4268.]
 [2100. 4358. 4522. 2346.]]
(4, 3, 2, 2)
dW[0,0,:,:]: [[258. 294.]
 [402. 438.]]
(1, 4)
db: [[ 36 117 198 279]]
